<a href="https://colab.research.google.com/github/divyasingh2611/RAG-for-tourism-/blob/main/Tour_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# importing Dependencies

In [ ]:
!pip install -U langchain-community
!pip install torch accelerate bitsandbytes transformers
!pip install langchain_openai
!pip install langchain
!pip install huggingface_hub

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install faiss-gpu

In [2]:
import langchain

from langchain.callbacks import StdOutCallbackHandler
from langchain.cache import InMemoryCache
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
import os

In [3]:
# loging an huggingface api key
from google.colab import userdata
api_key=userdata.get('Huggingface')

In [9]:
DB_FAISS_PATH = '/content/drive/MyDrive/tourism in india RAG/db_faiss'

custom_prompt_template = """As a knowledgeable and friendly travel and tour guide, your mission is to offer travelers the best advice and accurate information about their destinations.
Whether someone is seeking hidden gems, top attractions, or cultural insights, your role is to make their travel experience memorable and stress-free.
Answer all questions with clarity, enthusiasm, and a touch of local flavor, ensuring that travelers feel confident and excited about their journey.
Remember to include helpful tips and unique facts that will enrich their adventures.
Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model_4bit = AutoModelForCausalLM.from_pretrained(model_id,
                                                  device_map="auto",
                                                  quantization_config=quantization_config, )
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
pipeline = pipeline(
    "text-generation",
    model=model_4bit,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=200,
    max_new_tokens=500,
    do_sample=True,
    temperature=0.2,
)

In [ ]:
llm = HuggingFacePipeline(pipeline=pipeline)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [11]:
langchain.llm_cache = InMemoryCache()

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [13]:
DB_FAISS_PATH = '/content/drive/MyDrive/tourism in india RAG/db_faiss'

In [19]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                    model_kwargs={'device': 'cuda'})  # Use 'cuda' for GPU in Colab

In [22]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

handler = StdOutCallbackHandler()

db = FAISS.load_local(DB_FAISS_PATH,
                      embeddings,
                      allow_dangerous_deserialization=True)

# Retrieval QA Chain
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                        chain_type='stuff',
                                        retriever=db.as_retriever(search_kwargs={'k': 50}),
                                      # retriever=db.as_retriever(search_kwargs={'k': 2},
                                      # "search_type" : "similarity"}
                                        return_source_documents=True,
                                        verbose = True,
                                        callbacks=[handler],
                                        chain_type_kwargs={'prompt': set_custom_prompt()}
                                        )

In [23]:
def ask_and_answer(question):
  """
  As a knowledgeable and friendly travel and tour guide, your mission is to offer travelers the best advice and accurate information about their destinations. Whether someone is seeking hidden gems, top attractions, or cultural insights, your role is to make their travel experience memorable and stress-free. Answer all questions with clarity, enthusiasm, and a touch of local flavor, ensuring that travelers feel confident and excited about their journey. Remember to include helpful tips and unique facts that will enrich their adventures.

  """
  query =  question

  ans = qa_chain({"query": query})
  ans = ans['result']
  #print(f"Response generated:\n {ans['result']}")
  return ans
# # Example usage (assuming your qa_chain is defined elsewhere)
# answer=ask_and_answer(qa_chain)

In [24]:
question=input("Please Enter your Questions")

Please Enter your Questionssuggest me places to vistit in maharahstra


In [25]:
answer=ask_and_answer(question)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


In [26]:
print(answer)

Maharashtra, a great state in Western India, has many important cities such as Pune, Nasik, Mumbai, and Nagpur, each with its distinct identity. Here are some places to visit in Maharashtra:

1. Pune: Known for its vibrant student community and educational institutes. Visit the Aga Khan Palace, Shaniwar Wada, and Osho Ashram.

2. Nasik: Abode of the Maha Kumbh Mela, and home to ancient temples and caves. Don't miss Pandavleni Caves and Sula Vineyards.

3. Mumbai: Financial capital of India and the city of dreams. Explore the Gateway of India, Elephanta Caves, and Dhobi Ghat.

4. Nagpur: The orange city, and the winter capital of Maharashtra. Visit the Deekshabhoomi, Maharajbagh Zoo, and Raman Science Centre.

5. Rajasthan: Although not in Maharashtra, it's a must-visit destination near Maharashtra. Explore the palaces and forts of Jaipur, Jodhpur, Bikaner, and Jaisalmer, and enjoy the lakes of Udaipur.

6. Mount Abu: A popular hill station with many Jain temples of noteworthy importanc

# Creating interface

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
iface =gr.Interface(fn=ask_and_answer,
             inputs=gr.Textbox(label="Please Enter your Questions"),
             outputs=gr.Textbox(label="Response"),
             title="Your tour guide",
             live=False)
iface.launch(share=True)